<div style="text-align: center; background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%); 
            padding: 40px 30px; border-radius: 12px; color: #2c3e50;
            box-shadow: 0 10px 25px rgba(0,0,0,0.1); margin-bottom: 20px; font-family: 'Segoe UI', sans-serif;">
    <div style="max-width: 700px; margin: 0 auto;">
        <div style="margin-bottom: 20px;">
            <h1 style="color: #1a252f; font-size: 2.8em; margin: 0; 
                       font-weight: 800; letter-spacing: -1px; text-transform: uppercase;">
                Machine Learning Report
            </h1>
            <div style="height: 4px; width: 60px; background-color: #3498db; margin: 15px auto 0; border-radius: 2px;"></div>
        </div>
        <div>
            <div style="background-color: #ffffff; 
                        padding: 12px 30px; border-radius: 50px; 
                        display: inline-block; 
                        box-shadow: 0 4px 6px rgba(0,0,0,0.05);
                        border: 1px solid rgba(0,0,0,0.05);">
                <h3 style="color: #555; margin: 0; font-weight: 500; 
                           font-size: 1rem; letter-spacing: 0.5px;">
                    <span style="color: #3498db; font-weight: 700; text-transform: uppercase; font-size: 0.9em;">Author:</span> Hoang Son Lai 
                    <span style="margin: 0 10px; color: #ddd;">|</span>
                    <span style="color: #3498db; font-weight: 700; text-transform: uppercase; font-size: 0.9em;">Date:</span> 10/12/2025
                </h3>
            </div>
        </div>
    </div>
</div>